# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298415901604                   -0.85    5.0
  2   -8.300196359389       -2.75       -1.25    1.8
  3   -8.300432739612       -3.63       -1.87    2.0
  4   -8.300461406603       -4.54       -2.73    2.1
  5   -8.300464425480       -5.52       -3.17    3.0
  6   -8.300464539680       -6.94       -3.31   10.2
  7   -8.300464590786       -7.29       -3.47    2.8
  8   -8.300464616573       -7.59       -3.61    5.2
  9   -8.300464636859       -7.69       -3.81    1.4
 10   -8.300464639640       -8.56       -3.90    1.8
 11   -8.300464643498       -8.41       -4.20    1.8


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67445756490                   -0.70    6.4
  2   -16.67869268906       -2.37       -1.14    1.0
  3   -16.67919438572       -3.30       -1.87    1.9
  4   -16.67927567722       -4.09       -2.75    3.2
  5   -16.67928572032       -5.00       -3.16    7.5
  6   -16.67928617640       -6.34       -3.46    2.1
  7   -16.67928621345       -7.43       -3.91    1.4
  8   -16.67928622373       -7.99       -4.54    2.8


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32688125645                   -0.56    7.4
  2   -33.33433665773       -2.13       -1.00    1.6
  3   -33.33597215848       -2.79       -1.72    6.1
  4   -33.33614685565       -3.76       -2.62    3.9
  5   -33.33682716833       -3.17       -2.30    6.5
  6   -33.33686122689       -4.47       -2.38    2.6
  7   -33.33694368057       -4.08       -3.65    1.9
  8   -33.33694390217       -6.65       -3.79    5.6
  9   -33.33694392459       -7.65       -4.04    1.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298549941795                   -0.85    5.0
  2   -8.300276253906       -2.76       -1.59    1.0
  3   -8.300425433262       -3.83       -2.55    3.2
  4   -8.300379372160   +   -4.34       -2.29    5.2
  5   -8.300464278696       -4.07       -3.39    1.1
  6   -8.300464587925       -6.51       -3.83    3.5
  7   -8.300464640714       -7.28       -4.25    1.2


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32634446133                   -0.56    6.6
  2   -33.30762410553   +   -1.73       -1.26    1.8
  3   -7.521542111544   +    1.41       -0.36    6.1
  4   -33.31018932161        1.41       -1.54    5.1
  5   -33.24906466096   +   -1.21       -1.39    3.5
  6   -29.25204751894   +    0.60       -0.76    4.2
  7   -33.33417691847        0.61       -2.19    4.2
  8   -33.33374040891   +   -3.36       -2.17    3.2
  9   -33.33581458635       -2.68       -2.31    2.1
 10   -33.33616980059       -3.45       -2.59    1.8
 11   -33.33656701897       -3.40       -2.75    1.1
 12   -33.33689424423       -3.49       -3.19    2.9
 13   -33.33692134430       -4.57       -3.34    3.2
 14   -33.33692982859       -5.07       -3.45    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.